# **Q7**
Repeat Q6 using a 5-fold CV-based selection of training and test instances for each
model. Evaluate the accuracy, sensitivity, and specificity values of multiclass LoR,
multiclass LoR+L2-norm regularization, multiclass LoR+L1-norm regularization
models using BGD, SGD, and MBGD algorithms. Evaluate the performance of each
model using individual accuracy and overall accuracy measures. You must use the
dataset data_q6_q7.txt for this question.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from google.colab import files
uploaded=files.upload()

Saving data_q6_q7.txt to data_q6_q7.txt


In [ ]:
def hyp(x,w):
  return np.dot(x,w)
def hyp_lor(x,w):
  return 1/(1+np.exp(-(hyp(x,w))))
def mse(y,pred,l):
  return (1/l)*np.sum(np.square(y-pred))

In [ ]:
f,tr,va,te=8,147,21,42
data=pd.read_csv("/content/data_q6_q7.txt", delimiter = "\t", header=None)
data=pd.DataFrame(data)
data.columns=['a','b','c','d','e','f','g','class_name']
data.insert(0,"x0",1)
data.insert(f,"class_1",0);data.insert(f+1,"class_2",0);data.insert(f+2,"class_3",0)
dbool1,dbool2,dbool3 = (data.class_name==1),(data.class_name==2),(data.class_name==3)
data.loc[dbool1,'class_1'],data.loc[dbool2,'class_2'],data.loc[dbool3,'class_3'] = 1,1,1
data

,x0,a,b,c,d,e,f,g,class_1,class_2,class_3,class_name
0,1,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1,0,0,1
1,1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1,0,0,1
2,1,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1,0,0,1
3,1,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1,0,0,1
4,1,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
205,1,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,0,0,1,3
206,1,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,0,0,1,3
207,1,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,0,0,1,3
208,1,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,0,0,1,3


# one vs all

In [ ]:
data=np.array(data); np.random.shuffle(data);np.random.shuffle(data)
data=pd.DataFrame(data)

## BGD

### LoR

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w1)-y1_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w1[j] = w1[j] - (1/len(data_tr[:,f]))*2*jsum

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w2)-y2_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w2[j] = w2[j] - (1/len(data_tr[:,f]))*0.2*jsum

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w3)-y3_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w3[j] = w3[j] - (1/len(data_tr[:,f]))*1*jsum

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.75
Class 2 Accuracy = 0.9090909090909091
Class 3 Accuracy = 1.0
Overall Accuracy = 0.8604651162790697
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.8333333333333334
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.8604651162790697
RUN 3
Class 1 Accuracy = 1.0
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.875
Overall Accuracy = 0.9534883720930233
RUN 4
Class 1 Accuracy = 0.7647058823529411
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9473684210526315
Overall Accuracy = 0.8837209302325582
RUN 5
Class 1 Accuracy = 0.75
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8571428571428571
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.8129411764705882
Average Class 1 Accuracy = 0.9484848484848485
Average Class 1 Accuracy = 0.911140350877193
Average Overall Accuracy = 0.8830564784053155


### LoR with L2 NORM

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w1)-y1_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w1[j] = (1-(0.001*1.25))*w1[j] - (1/len(data_tr[:,f]))*1.25*jsum

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w2)-y2_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w2[j] = (1-(0.01*0.3))*w2[j] - (1/len(data_tr[:,f]))*0.3*jsum

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w3)-y3_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w3[j] = (1-(0.01*0.3))*w3[j] - (1/len(data_tr[:,f]))*0.3*jsum

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.8
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9166666666666666
Overall Accuracy = 0.8837209302325582
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.7222222222222222
Class 3 Accuracy = 0.8666666666666667
Overall Accuracy = 0.7906976744186046
RUN 3
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.9473684210526315
Class 3 Accuracy = 0.8125
Overall Accuracy = 0.9069767441860465
RUN 4
Class 1 Accuracy = 0.7058823529411765
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8947368421052632
Overall Accuracy = 0.8372093023255814
RUN 5
Class 1 Accuracy = 0.6875
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8333333333333334
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.7986764705882353
Average Class 1 Accuracy = 0.9339181286549707
Average Class 1 Accuracy = 0.8581140350877193
Average Overall Accuracy = 0.8503875968992247


### LoR with L1 NORM

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w1)-y1_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w1[j] = w1[j] - (1/len(data_tr[:,f]))*0.1*jsum - ((0.1*0.05)/2)*np.sign(w1[j])

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w2)-y2_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w2[j] = w2[j] - (1/len(data_tr[:,f]))*1*jsum - ((1*0.005)/2)*np.sign(w2[j])

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x_train,w3)-y3_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(len(data_tr[:,f]),1)))
      w3[j] = w3[j] - (1/len(data_tr[:,f]))*0.1*jsum - ((0.1*0.05)/2)*np.sign(w3[j])

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.6
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.813953488372093
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.9444444444444444
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.9069767441860465
RUN 3
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.9473684210526315
Class 3 Accuracy = 0.75
Overall Accuracy = 0.8837209302325582
RUN 4
Class 1 Accuracy = 0.8235294117647058
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9473684210526315
Overall Accuracy = 0.9069767441860465
RUN 5
Class 1 Accuracy = 0.6875
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8333333333333334
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.7822058823529412
Average Class 1 Accuracy = 0.9783625730994153
Average Class 1 Accuracy = 0.886140350877193
Average Overall Accuracy = 0.8689922480620155


## SGD

### LoR

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(1000):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w1)[index]-y1_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w1[j] = w1[j] - 0.3*j2

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(1000):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w2)[index]-y2_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w2[j] = w2[j] - 0.7*j2

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(1000):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w3)[index]-y3_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w3[j] = w3[j] - 0.7*j2

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.55
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.7906976744186046
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.5
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.7209302325581395
RUN 3
Class 1 Accuracy = 0.5
Class 2 Accuracy = 0.5789473684210527
Class 3 Accuracy = 1.0
Overall Accuracy = 0.7209302325581395
RUN 4
Class 1 Accuracy = 0.5294117647058824
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9473684210526315
Overall Accuracy = 0.7906976744186046
RUN 5
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8809523809523809
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.6383823529411765
Average Class 1 Accuracy = 0.8157894736842106
Average Class 1 Accuracy = 0.936140350877193
Average Overall Accuracy = 0.7808416389811739


### LoR with L2 NORM

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(250):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w1)[index]-y1_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w1[j] = (1-(0.0004*1.3))*w1[j] - 1.3*j2

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(370):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w2)[index]-y2_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w2[j] = (1-(0.009*1.2))*w2[j] - 1.2*j2

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(370):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w3)[index]-y3_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w3[j] = (1-(0.008*2))*w3[j] - 2*j2

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.25
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.5
Overall Accuracy = 0.5116279069767442
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.7222222222222222
Class 3 Accuracy = 0.6
Overall Accuracy = 0.6976744186046512
RUN 3
Class 1 Accuracy = 0.0
Class 2 Accuracy = 0.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.37209302325581395
RUN 4
Class 1 Accuracy = 0.5294117647058824
Class 2 Accuracy = 0.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.6511627906976745
RUN 5
Class 1 Accuracy = 0.5
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.0
Overall Accuracy = 0.5714285714285714
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.4158823529411765
Average Class 1 Accuracy = 0.5444444444444445
Average Class 1 Accuracy = 0.62
Average Overall Accuracy = 0.5607973421926911


### LoR with L1 NORM

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(250):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w1)[index]-y1_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w1[j] = w1[j] - 0.1*j2 - ((0.1*0.005)/2)*np.sign(w1[j])

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(370):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w2)[index]-y2_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w2[j] = w2[j] - 0.1*j2 - ((0.1*0.005)/2)*np.sign(w2[j])

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(370):
    index=np.random.randint(len(data_tr[:,f]))
    j1 = hyp_lor(x_train,w3)[index]-y3_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
      w3[j] = w3[j] - 0.1*j2 - ((0.1*0.005)/2)*np.sign(w3[j])

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.35
Class 2 Accuracy = 0.6363636363636364
Class 3 Accuracy = 1.0
Overall Accuracy = 0.6046511627906976
RUN 2
Class 1 Accuracy = 0.0
Class 2 Accuracy = 0.9444444444444444
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.7209302325581395
RUN 3
Class 1 Accuracy = 0.0
Class 2 Accuracy = 0.5789473684210527
Class 3 Accuracy = 1.0
Overall Accuracy = 0.627906976744186
RUN 4
Class 1 Accuracy = 0.0
Class 2 Accuracy = 0.8571428571428571
Class 3 Accuracy = 1.0
Overall Accuracy = 0.5813953488372093
RUN 5
Class 1 Accuracy = 0.25
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.7
Overall Accuracy = 0.6428571428571429
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.12
Average Class 1 Accuracy = 0.8033796612743981
Average Class 1 Accuracy = 0.9266666666666667
Average Overall Accuracy = 0.6355481727574751


## MBGD

### LoR

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w1) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w1[j] = w1[j] - (1/15)*1.5*jsum

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w2) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w2[j] = w2[j] - (1/15)*0.7*jsum

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w3) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w3[j] = w3[j] - (1/15)*0.8*jsum

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.65
Class 2 Accuracy = 0.8181818181818182
Class 3 Accuracy = 1.0
Overall Accuracy = 0.7906976744186046
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.8888888888888888
Class 3 Accuracy = 0.8666666666666667
Overall Accuracy = 0.8604651162790697
RUN 3
Class 1 Accuracy = 1.0
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8125
Overall Accuracy = 0.9302325581395349
RUN 4
Class 1 Accuracy = 0.9411764705882353
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8421052631578947
Overall Accuracy = 0.9069767441860465
RUN 5
Class 1 Accuracy = 0.6875
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8333333333333334
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.8157352941176471
Average Class 1 Accuracy = 0.9414141414141415
Average Class 1 Accuracy = 0.8642543859649123
Average Overall Accuracy = 0.8643410852713178


### LoR with L2 NORM

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w1) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w1[j] = (1-(0.001*1))*w1[j] - 1*jsum

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w2) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w2[j] = (1-(0.008*1))*w2[j] - 1*jsum

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w3) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w3[j] = (1-(0.008*0.4))*w3[j] - 0.4*jsum

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.95
Class 2 Accuracy = 0.6363636363636364
Class 3 Accuracy = 0.75
Overall Accuracy = 0.813953488372093
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.5555555555555556
Class 3 Accuracy = 0.6666666666666666
Overall Accuracy = 0.6511627906976745
RUN 3
Class 1 Accuracy = 0.0
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.5
Overall Accuracy = 0.627906976744186
RUN 4
Class 1 Accuracy = 0.6470588235294118
Class 2 Accuracy = 0.0
Class 3 Accuracy = 0.9473684210526315
Overall Accuracy = 0.6744186046511628
RUN 5
Class 1 Accuracy = 0.0
Class 2 Accuracy = 0.9375
Class 3 Accuracy = 1.0
Overall Accuracy = 0.5952380952380952
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.4794117647058823
Average Class 1 Accuracy = 0.6258838383838384
Average Class 1 Accuracy = 0.7728070175438596
Average Overall Accuracy = 0.6725359911406423


### LoR with L1 NORM

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  data_te = data.loc[start:end,:].copy()
  data_tr = data.drop(data.index[start:end]).copy() ; data_tr = data_tr.loc[:,:f+2]
  data_tr=np.array(data_tr) ; data_te=np.array(data_te)
  norm = np.amax(np.abs(data_tr),axis=0)
  data_tr,data_te_nor = data_tr/norm,data_te[:,:-1]/norm #data normalised

  x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(len(data_te[:,f+3]),1)
  x_train = data_tr[:,:f]

  y1_train = data_tr[:,f].reshape(len(data_tr[:,f]),1)
  y2_train = data_tr[:,f+1].reshape(len(data_tr[:,f+1]),1)
  y3_train = data_tr[:,f+2].reshape(len(data_tr[:,f+2]),1)

  # 1 vs (2,3)

  w1=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w1) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w1[j] = w1[j] - (1/15)*1*jsum - ((1*0.05)/2)*np.sign(w1[j])

  # 2 vs (1,3)

  w2=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w2) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w2[j] = w2[j] - (1/15)*0.8*jsum - ((0.8*0.005)/2)*np.sign(w2[j])

  # 3 vs (1,2)

  w3=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(data_tr)
    hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w3) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,data_tr[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
      w3[j] = w3[j] - (1/15)*0.5*jsum - ((0.5*0.005)/2)*np.sign(w3[j])

  # testing

  y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
  output=np.zeros((len(data_te[:,f+3]),1))

  for i in range(len(data_te[:,f+3])):
    class_def=np.argmax(y_pred[i])
    output[i]=class_def+1

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.75
Class 2 Accuracy = 0.9090909090909091
Class 3 Accuracy = 0.9166666666666666
Overall Accuracy = 0.8372093023255814
RUN 2
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.7777777777777778
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.8372093023255814
RUN 3
Class 1 Accuracy = 0.0
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.813953488372093
RUN 4
Class 1 Accuracy = 0.8823529411764706
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8947368421052632
Overall Accuracy = 0.9069767441860465
RUN 5
Class 1 Accuracy = 0.75
Class 2 Accuracy = 0.875
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8095238095238095
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.6364705882352941
Average Class 1 Accuracy = 0.9123737373737374
Average Class 1 Accuracy = 0.9089473684210526
Average Overall Accuracy = 0.8409745293466223


# one vs one

### data splitting

In [ ]:
data=np.array(data); np.random.shuffle(data);np.random.shuffle(data)

## LoR

### BGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x12_train,w12)-y12_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x12_train[:,j].reshape(len(y12_train),1)))
      w12[j] = w12[j] - (1/len(y12_train))*0.3*jsum

  # 2 vs 3

  w23=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x23_train,w23)-y23_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x23_train[:,j].reshape(len(y23_train),1)))
      w23[j] = w23[j] - (1/len(y23_train))*0.3*jsum

  # 3 vs 1
  w31=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x31_train,w31)-y31_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x31_train[:,j].reshape(len(y31_train),1)))
      w31[j] = w31[j] - (1/len(y31_train))*0.8*jsum

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.9047619047619048
RUN 2
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.875
Class 3 Accuracy = 0.9090909090909091
Overall Accuracy = 0.9285714285714286
RUN 3
Class 1 Accuracy = 0.9
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.9047619047619048
RUN 4
Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.8571428571428571
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9047619047619048
RUN 5
Class 1 Accuracy = 0.6153846153846154
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 0.8235294117647058
Overall Accuracy = 0.7857142857142857
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.8405769230769231
Average Class 1 Accuracy = 0.9297619047619048
Average Class 1 Accuracy = 0.8931907308377897
Average Overall Accuracy = 0.8857142857142858


### SGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    index=np.random.randint(len(y12_train))
    j1 = hyp_lor(x12_train,w12)[index]-y12_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x12_train[index,j].reshape(1,1))
      w12[j] = w12[j] - 0.3*j2

  # 2 vs 1

  w23=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    index=np.random.randint(len(y23_train))
    j1 = hyp_lor(x23_train,w23)[index]-y23_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x23_train[index,j].reshape(1,1))
      w23[j] = w23[j] - 0.7*j2

  # 3 vs 1

  w31=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    index=np.random.randint(len(y31_train))
    j1 = hyp_lor(x31_train,w31)[index]-y31_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x31_train[index,j].reshape(1,1))
      w31[j] = w31[j] - 0.7*j2

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.6875
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.8809523809523809
RUN 2
Class 1 Accuracy = 0.8666666666666667
Class 2 Accuracy = 0.4375
Class 3 Accuracy = 1.0
Overall Accuracy = 0.7380952380952381
RUN 3
Class 1 Accuracy = 0.9
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.9047619047619048
RUN 4
Class 1 Accuracy = 0.5625
Class 2 Accuracy = 0.21428571428571427
Class 3 Accuracy = 1.0
Overall Accuracy = 0.5714285714285714
RUN 5
Class 1 Accuracy = 0.46153846153846156
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.7647058823529411
Overall Accuracy = 0.7380952380952381
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.6956410256410257
Average Class 1 Accuracy = 0.7303571428571429
Average Class 1 Accuracy = 0.9129411764705881
Average Overall Accuracy = 0.7666666666666666


### MBGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train12)
    tr12 = train12/norm_mb
    hypo1 = np.dot(tr12[:15,:f].reshape(15,f),w12) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr12[:15,f].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr12[:15,j].reshape(15,1)))
      w12[j] = w12[j] - (1/15)*0.6*jsum

  # 2 vs 3

  w23=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train23)
    tr23 = train23/norm_mb
    hypo1 = np.dot(tr23[:15,:f].reshape(15,f),w23) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr23[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr23[:15,j].reshape(15,1)))
      w23[j] = w23[j] - (1/15)*0.7*jsum

  # 3 vs 1

  w31=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train31)
    tr31 = train31/norm_mb
    hypo1 = np.dot(tr31[:15,:f].reshape(15,f),w31) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr31[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr31[:15,j].reshape(15,1)))
      w31[j] = w31[j] - (1/15)*0.8*jsum

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.8181818181818182
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.8809523809523809
RUN 2
Class 1 Accuracy = 0.9333333333333333
Class 2 Accuracy = 0.9375
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9523809523809523
RUN 3
Class 1 Accuracy = 0.8
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8809523809523809
RUN 4
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 0.8571428571428571
Class 3 Accuracy = 1.0
Overall Accuracy = 0.8809523809523809
RUN 5
Class 1 Accuracy = 0.38461538461538464
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8235294117647058
Overall Accuracy = 0.7380952380952381
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.7610897435897436
Average Class 1 Accuracy = 0.9225649350649352
Average Class 1 Accuracy = 0.9113725490196078
Average Overall Accuracy = 0.8666666666666668


## LoR with L2 NORM

### BGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x12_train,w12)-y12_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x12_train[:,j].reshape(len(y12_train),1)))
      w12[j] = (1-(0.001*2))*w12[j] - (1/len(y12_train))*2*jsum

  # 2 vs 3

  w23=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x23_train,w23)-y23_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x23_train[:,j].reshape(len(y23_train),1)))
      w23[j] = (1-(0.002*1))*w23[j] - (1/len(y23_train))*1*jsum

  # 3 vs 1
  w31=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x31_train,w31)-y31_train # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x31_train[:,j].reshape(len(y31_train),1)))
      w31[j] = (1-(0.002*1))*w31[j] - (1/len(y31_train))*1*jsum

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 0.9090909090909091
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.8809523809523809
RUN 2
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.9375
Class 3 Accuracy = 0.9090909090909091
Overall Accuracy = 0.9523809523809523
RUN 3
Class 1 Accuracy = 0.9
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.9047619047619048
RUN 4
Class 1 Accuracy = 0.9375
Class 2 Accuracy = 0.9285714285714286
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9523809523809523
RUN 5
Class 1 Accuracy = 0.6153846153846154
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8235294117647058
Overall Accuracy = 0.8095238095238095
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.8530769230769231
Average Class 1 Accuracy = 0.9550324675324676
Average Class 1 Accuracy = 0.8931907308377897
Average Overall Accuracy = 0.9


### SGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(600):
    index=np.random.randint(len(y12_train))
    j1 = hyp_lor(x12_train,w12)[index]-y12_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x12_train[index,j].reshape(1,1))
      w12[j] = (1-(0.0001*1))*w12[j] - 1*j2

  # 2 vs 1

  w23=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    index=np.random.randint(len(y23_train))
    j1 = hyp_lor(x23_train,w23)[index]-y23_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x23_train[index,j].reshape(1,1))
      w23[j] = (1-(0.0001*1))*w23[j] - 1*j2

  # 3 vs 1

  w31=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    index=np.random.randint(len(y31_train))
    j1 = hyp_lor(x31_train,w31)[index]-y31_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x31_train[index,j].reshape(1,1))
      w31[j] = (1-(0.0001*1))*w31[j] - 1*j2

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.625
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.8571428571428571
RUN 2
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.9375
Class 3 Accuracy = 0.8181818181818182
Overall Accuracy = 0.9285714285714286
RUN 3
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.058823529411764705
Class 3 Accuracy = 0.06666666666666667
Overall Accuracy = 0.2857142857142857
RUN 4
Class 1 Accuracy = 0.75
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9047619047619048
RUN 5
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.8333333333333334
Class 3 Accuracy = 0.8235294117647058
Overall Accuracy = 0.8809523809523809
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.875
Average Class 1 Accuracy = 0.7659313725490196
Average Class 1 Accuracy = 0.7416755793226381
Average Overall Accuracy = 0.7714285714285714


### MBGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train12)
    tr12 = train12/norm_mb
    hypo1 = np.dot(tr12[:5,:f].reshape(5,f),w12) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr12[:5,f].reshape(5,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr12[:5,j].reshape(5,1)))
      w12[j] = (1-(0.002*1))*w12[j] - (1/5)*1*jsum

  # 2 vs 3

  w23=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train23)
    tr23 = train23/norm_mb
    hypo1 = np.dot(tr23[:5,:f].reshape(5,f),w23) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr23[:5,f+1].reshape(5,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr23[:5,j].reshape(5,1)))
      w23[j] = (1-(0.002*1))*w23[j] - (1/5)*1*jsum

  # 3 vs 1

  w31=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train31)
    tr31 = train31/norm_mb
    hypo1 = np.dot(tr31[:5,:f].reshape(5,f),w31) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr31[:5,f+2].reshape(5,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr31[:5,j].reshape(5,1)))
      w31[j] = (1-(0.002*1))*w31[j] - (1/5)*1*jsum

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286
RUN 2
Class 1 Accuracy = 0.7333333333333333
Class 2 Accuracy = 0.6875
Class 3 Accuracy = 1.0
Overall Accuracy = 0.7857142857142857
RUN 3
Class 1 Accuracy = 0.9
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.9047619047619048
RUN 4
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 0.6428571428571429
Class 3 Accuracy = 1.0
Overall Accuracy = 0.8095238095238095
RUN 5
Class 1 Accuracy = 0.38461538461538464
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8235294117647058
Overall Accuracy = 0.7380952380952381
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.7285897435897436
Average Class 1 Accuracy = 0.8660714285714285
Average Class 1 Accuracy = 0.924705882352941
Average Overall Accuracy = 0.8333333333333334


## LoR with L1 NORM

### BGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x12_train,w12)-y12_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x12_train[:,j].reshape(len(y12_train),1)))
      w12[j] = w12[j] - (1/len(y12_train))*2*jsum - ((2*0.0005)/2)*np.sign(w12[j])

  # 2 vs 3

  w23=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x23_train,w23)-y23_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x23_train[:,j].reshape(len(y23_train),1)))
      w23[j] = w23[j] - (1/len(y23_train))*0.3*jsum - ((0.3*0.005)/2)*np.sign(w23[j])

  # 3 vs 1
  w31=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(500):
    j1 = hyp_lor(x31_train,w31)-y31_train, # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,x31_train[:,j].reshape(len(y31_train),1)))
      w31[j] = w31[j] - (1/len(y31_train))*0.8*jsum - ((0.8*0.005)/2)*np.sign(w31[j])

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 0.9090909090909091
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.8809523809523809
RUN 2
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.875
Class 3 Accuracy = 0.9090909090909091
Overall Accuracy = 0.9285714285714286
RUN 3
Class 1 Accuracy = 0.9
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8
Overall Accuracy = 0.9047619047619048
RUN 4
Class 1 Accuracy = 0.9375
Class 2 Accuracy = 0.8571428571428571
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286
RUN 5
Class 1 Accuracy = 0.6923076923076923
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8235294117647058
Overall Accuracy = 0.8333333333333334
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.8684615384615384
Average Class 1 Accuracy = 0.9282467532467532
Average Class 1 Accuracy = 0.8931907308377897
Average Overall Accuracy = 0.8952380952380953


### SGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(1000):
    index=np.random.randint(len(y12_train))
    j1 = hyp_lor(x12_train,w12)[index]-y12_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x12_train[index,j].reshape(1,1))
      w12[j] = w12[j] - 0.7*j2 - ((0.7*0.005)/2)*np.sign(w12[j])

  # 2 vs 1

  w23=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(1000):
    index=np.random.randint(len(y23_train))
    j1 = hyp_lor(x23_train,w23)[index]-y23_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x23_train[index,j].reshape(1,1))
      w23[j] = w23[j] - 0.7*j2 - ((0.7*0.0005)/2)*np.sign(w23[j])

  # 3 vs 1

  w31=np.random.rand(f).reshape(f,1) # weight vector initialised
  for t in range(1000):
    index=np.random.randint(len(y31_train))
    j1 = hyp_lor(x31_train,w31)[index]-y31_train[index] # (hyp(xi)-yi)
    for j in range(f):
      j2 = np.multiply(j1,x31_train[index,j].reshape(1,1))
      w31[j] = w31[j] - 0.7*j2 - ((0.7*0.005)/2)*np.sign(w31[j])

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.75
Class 2 Accuracy = 0.5454545454545454
Class 3 Accuracy = 1.0
Overall Accuracy = 0.7857142857142857
RUN 2
Class 1 Accuracy = 0.8666666666666667
Class 2 Accuracy = 0.9375
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286
RUN 3
Class 1 Accuracy = 0.8
Class 2 Accuracy = 0.9411764705882353
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8571428571428571
RUN 4
Class 1 Accuracy = 0.6875
Class 2 Accuracy = 0.9285714285714286
Class 3 Accuracy = 1.0
Overall Accuracy = 0.8571428571428571
RUN 5
Class 1 Accuracy = 0.3076923076923077
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.8823529411764706
Overall Accuracy = 0.7380952380952381
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.6823717948717949
Average Class 1 Accuracy = 0.8705404889228419
Average Class 1 Accuracy = 0.9364705882352939
Average Overall Accuracy = 0.8333333333333334


### MBGD

In [ ]:
class_acc,o_acc=[0,0,0],0
for i in range(5):

  # data manipulation
  run=i
  print('RUN '+str(i+1))
  start,end=i*42,(i+1)*42
  x_train = np.delete(data, slice(start,end), axis=0).copy() ; y_train = x_train[:,-1].copy()
  norm = np.amax(np.abs(x_train[:,:f]),axis=0)
  x_test = data[start:end,:f].copy()/norm ; y_test = data[start:end,-1].copy().reshape(42,1) ; te=42
  class_1 = x_train[np.where(y_train==1)]
  class_2 = x_train[np.where(y_train==2)]
  class_3 = x_train[np.where(y_train==3)]
  train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
  train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
  train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
  y12_train = train12[:,f].reshape(len(train12[:,f]),1)
  y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
  y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
  x12_train = train12[:,:f].copy()/norm
  x23_train = train23[:,:f].copy()/norm
  x31_train = train31[:,:f].copy()/norm
  norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)
  return ([x12_train,y12_train,x23_train,y23_train,x31_train,y31_train,x_test,y_test,train12,train23,train31,norm_mb])

  # 1 vs 2

  w12=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train12)
    tr12 = train12/norm_mb
    hypo1 = np.dot(tr12[:15,:f].reshape(15,f),w12) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr12[:15,f].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr12[:15,j].reshape(15,1)))
      w12[j] = w12[j] - (1/15)*0.6*jsum - ((0.6*0.005)/2)*np.sign(w12[j])

  # 2 vs 3

  w23=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train23)
    tr23 = train23/norm_mb
    hypo1 = np.dot(tr23[:15,:f].reshape(15,f),w23) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr23[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr23[:15,j].reshape(15,1)))
      w23[j] = w23[j] - (1/15)*0.7*jsum - ((0.7*0.0025)/2)*np.sign(w23[j])

  # 3 vs 1

  w31=np.random.rand(f).reshape(f,1)
  for t in range(450):
    np.random.shuffle(train31)
    tr31 = train31/norm_mb
    hypo1 = np.dot(tr31[:15,:f].reshape(15,f),w31) # hypothesis array part1
    hypo = 1/(1+np.exp(-hypo1))
    j1 = np.subtract(hypo,tr31[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
    for j in range(f):
      jsum = np.sum(np.multiply(j1,tr31[:15,j].reshape(15,1)))
      w31[j] = w31[j] - (1/15)*0.8*jsum - ((0.8*0.005)/2)*np.sign(w31[j])

  # testing

  te=len(y_test)
  y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
  output=np.zeros((te,1))

  for i in range(te):
    y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
    y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
    y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
    class_def=stats.mode(y_pred[i])
    output[i]=class_def[0]

  comparison=np.concatenate((output,y_test),axis=1)

  # performance evaluation

  conf_matrix=np.zeros((3,3)) # confusion matrix
  for comp in comparison:
    pred,act=int(comp[0]-1),int(comp[1]-1)
    conf_matrix[act][pred]+=1

  cm,o_num,o_den=conf_matrix,0,0
  for i in range(3):
    den=0
    for j in range(3):
      den=den+cm[i][j]
    acc=cm[i][i]/den
    print('Class {} Accuracy = {}'.format(i+1,acc))
    class_acc[i] = class_acc[i] + acc
    o_num=o_num+cm[i][i]
    o_den=o_den+den

  print('Overall Accuracy = {}'.format(o_num/o_den))
  o_acc = o_acc + (o_num/o_den)

print('PERFORMANCE EVALUTATION')
print('Average Class 1 Accuracy = {}'.format(class_acc[0]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[1]/5))
print('Average Class 1 Accuracy = {}'.format(class_acc[2]/5))
print('Average Overall Accuracy = {}'.format(o_acc/5))

RUN 1
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.9047619047619048
RUN 2
Class 1 Accuracy = 1.0
Class 2 Accuracy = 0.875
Class 3 Accuracy = 0.9090909090909091
Overall Accuracy = 0.9285714285714286
RUN 3
Class 1 Accuracy = 0.9
Class 2 Accuracy = 0.7647058823529411
Class 3 Accuracy = 0.8
Overall Accuracy = 0.8095238095238095
RUN 4
Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.9285714285714286
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286
RUN 5
Class 1 Accuracy = 0.5384615384615384
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.7647058823529411
Overall Accuracy = 0.7619047619047619
PERFORMANCE EVALUTATION
Average Class 1 Accuracy = 0.8251923076923078
Average Class 1 Accuracy = 0.913655462184874
Average Class 1 Accuracy = 0.8814260249554368
Average Overall Accuracy = 0.8666666666666668
